In [1]:
# Apri il file in modalità lettura
with open('result_document.md', 'r') as file:
    # Leggi il contenuto del file
    contenuto = file.read()

# Stampa il contenuto del file
print(contenuto)

## Not with my name! Inferring artists' names of input strings employed by Diffusion Models

Roberto Leotta 1[0000 -0003 -4441 -8313] , Oliver Giudice 2[0000 -0002 -8343 -2049] , Luca Guarnera 3[0000 -0001 -8315 -351 X ] , and Sebastiano Battiato 1 3[0000 , -0001 -6127 -2470]

- 1 iCTLab Spinoff of University of Catania, Italy
- 3 Department of Mathematics and Computer Science, University of Catania, Italy roberto.leotta@ictlab.srl , oliver.giudice@bancaditalia.it , {luca.guarnera, sebastiano.battiato}@unict.it
- 2 Applied Research Team, IT dept., Banca d'Italia, Italy

Abstract. Diffusion Models (DM) are highly effective at generating realistic, high-quality images. However, these models lack creativity and merely compose outputs based on their training data, guided by a textual input provided at creation time. Is it acceptable to generate images reminiscent of an artist, employing his name as input? This imply that if the DM is able to replicate an artist's work then it was trained o

In [2]:
# 1. Pulizia del testo
import re
def clean_text(raw_text):
    text = raw_text
    text = re.sub(r'\s+', ' ', text)  # rimuove spazi multipli
    text = re.sub(r'[^\w\s.,;!?]', '', text)  # rimuove simboli speciali
    return text.strip()

clean_text(contenuto)
print('done')

done


In [3]:
# 2. Tokenizzazione
import nltk
#conda install -c conda-forge nltk

# Definisci un percorso specifico per i dati
nltk.data.path.append("/storage/data_4T_b/andreacutuli/PROVA/nome_ambiente/nltk_data")

# Scarica il corpus 'punkt'
nltk.download('punkt', download_dir="/storage/data_4T_b/andreacutuli/PROVA/nome_ambiente/nltk_data")
nltk.download('punkt_tab', download_dir="/storage/data_4T_b/andreacutuli/PROVA/nome_ambiente/nltk_data")


[nltk_data] Downloading package punkt to /storage/data_4T_b/andreacutu
[nltk_data]     li/PROVA/nome_ambiente/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /storage/data_4T_b/andrea
[nltk_data]     cutuli/PROVA/nome_ambiente/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
import nltk
#nltk.download('punkt')
from nltk.tokenize import sent_tokenize

sentences = sent_tokenize(clean_text(contenuto))


In [5]:
#3. Raggruppamento per similarità semantica
#!conda install -c conda-forge sentence-transformers
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(sentences)

# clustering con soglia o algoritmo
clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=0.5)
labels = clustering.fit_predict(embeddings)

# creare i chunk
from collections import defaultdict
chunks = defaultdict(list)
for sentence, label in zip(sentences, labels):
    chunks[label].append(sentence)

chunks = [" ".join(chunk) for chunk in chunks.values()]


Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.



/storage/data_4T_b/andreacutuli/miniconda3/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
print("sentence-transformers è stato installato correttamente!")


sentence-transformers è stato installato correttamente!


In [12]:
import random

# Seleziona un chunk a caso
random_chunk = random.choice(chunks)

# Stampa il chunk selezionato
print(random_chunk)


Several studies and articles have explored the legal ramifications of generative models and their potential to infringe on copyrights 5 .


In [16]:
import torch
print(torch.cuda.is_available())


False


In [14]:
#ASSEGNARE ETICHETTE AI CHUNK DA ETICHETTE PREDIFINITE

from transformers import pipeline

# Carica il modello zero-shot classification pre-addestrato
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


# Etichette generiche che il modello potrebbe utilizzare per classificare automaticamente i chunk
candidate_labels = ["paint", "model", "artist", "nature"]

# Assegna l'etichetta per ciascun chunk
chunk_labels = []
for chunk in chunks:
    result = classifier(chunk, candidate_labels)
    # Prendi l'etichetta con la probabilità più alta
    chunk_labels.append(result['labels'][0])

# Visualizza i risultati
for chunk, label in zip(chunks, chunk_labels):
    print(f"Chunk: {chunk}")
    print(f"Etichetta assegnata: {label}")
    print("-" * 40)


Device set to use cpu


Chunk: Not with my name!
Etichetta assegnata: artist
----------------------------------------
Chunk: Inferring artists names of input strings employed by Diffusion Models Roberto Leotta 10000 0003 4441 8313 , Oliver Giudice 20000 0002 8343 2049 , Luca Guarnera 30000 0001 8315 351 X  , and Sebastiano Battiato 1 30000 , 0001 6127 2470  1 iCTLab Spinoff of University of Catania, Italy  3 Department of Mathematics and Computer Science, University of Catania, Italy roberto.leottaictlab.srl , oliver.giudicebancaditalia.it , luca.guarnera, sebastiano.battiatounict.it  2 Applied Research Team, IT dept., Banca dItalia, Italy Abstract.
Etichetta assegnata: artist
----------------------------------------
Chunk: Diffusion Models DM are highly effective at generating realistic, highquality images.
Etichetta assegnata: model
----------------------------------------
Chunk: However, these models lack creativity and merely compose outputs based on their training data, guided by a textual input provided

In [ ]:
#1: Clustering tematico con K-means


from sklearn.cluster import KMeans
import numpy as np

# Usa le embedding dei chunk (da Sentence-BERT)
# embeddings è già stato calcolato in precedenza
embeddings = model.encode(chunks)  # Assicurati che chunks contenga i tuoi chunk

# Definisci il numero di cluster
num_clusters = 5  # Puoi cambiare questo numero in base al tuo caso

# Applica K-means
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
labels = kmeans.fit_predict(embeddings)

# Assegna un'etichetta al chunk
clustered_chunks = {}
for label, chunk in zip(labels, chunks):
    if label not in clustered_chunks:
        clustered_chunks[label] = []
    clustered_chunks[label].append(chunk)

# Visualizza i chunk per ciascun cluster
for cluster, texts in clustered_chunks.items():
    print(f"Cluster {cluster}:")
    for text in texts:
        print(f"- {text}")
    print("-" * 40)


In [ ]:
import gensim
from gensim import corpora
from nltk.tokenize import word_tokenize
import string

# Pulizia del testo (simile a quanto fatto prima)
def clean_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    return text

# Preprocessamento del testo (tokenizzazione)
tokenized_chunks = [word_tokenize(clean_text(chunk)) for chunk in chunks]

# Creazione del dizionario e del corpus
dictionary = corpora.Dictionary(tokenized_chunks)
corpus = [dictionary.doc2bow(text) for text in tokenized_chunks]

# Creazione del modello LDA
lda_model = gensim.models.LdaMulticore(corpus, num_topics=5, id2word=dictionary, passes=10)

# Visualizza i topic generati
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")
    print("-" * 40)

# Assegna i topic ai chunk
for i, chunk in enumerate(chunks):
    print(f"Chunk {i}: {chunk}")
    print(f"Topic: {lda_model.get_document_topics(corpus[i])}")
    print("-" * 40)


ImportError: cannot import name 'triu' from 'scipy.linalg.special_matrices' (/storage/data_4T_b/andreacutuli/miniconda3/envs/lda-env/lib/python3.10/site-packages/scipy/linalg/special_matrices.py)

In [ ]:
import gensim
